# テキストデータの可視化

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_5/intro-to-visualizing-text-data.ipynb)

今回は、[青空文庫](https://www.aozora.gr.jp/)に収録されている、夏目漱石の[『三四郎』](https://www.aozora.gr.jp/cards/000148/card794.html)という作品を使って、テキストデータの可視化を練習します。

## 基礎的な解析

In [1]:
# ライブラリのインストール

!pip install ginza==4.0.6 ja-ginza==4.0.0
!pip install sklearn pandas


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# ファイルをダウンロードする
!wget https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
# textフォルダ作る
!mkdir -p text
# ファイルをtextフォルダに解凍
!unzip -d text -o 794_ruby_4237.zip

--2022-11-09 15:16:51--  https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
Resolving proxy.noc.titech.ac.jp (proxy.noc.titech.ac.jp)... 131.112.125.238
Connecting to proxy.noc.titech.ac.jp (proxy.noc.titech.ac.jp)|131.112.125.238|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 158711 (155K) [application/zip]
Saving to: ‘794_ruby_4237.zip’

794_ruby_4237.zip   100%[===================>] 154.99K  --.-KB/s    in 0.01s   

2022-11-09 15:16:51 (14.1 MB/s) - ‘794_ruby_4237.zip’ saved [158711/158711]

Archive:  794_ruby_4237.zip
  inflating: text/sanshiro.txt       


正規表現で青空文庫のルビ、注、アクセントの記号を取り除きます。

同時に、文字コードをShift-JISからUTF-8にします。

```

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）頓狂《とんきょう》

｜：ルビの付く文字列の始まりを特定する記号
（例）福岡県｜京都郡《みやこぐん》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
　　　（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
（例）※［＃「魚＋師のつくり」、第4水準2-93-37］

〔〕：アクセント分解された欧文をかこむ
（例）〔ve'rite'《ヴェリテ》 vraie《ヴレイ》.〕
アクセント分解についての詳細は下記URLを参照してください
http://www.aozora.gr.jp/accent_separation.html
-------------------------------------------------------

```

In [5]:
import re

input_fn = "text/sanshiro.txt"
output_fn = "text/sanshiro.stripruby.txt"

with open(input_fn, encoding="shift_jis") as fin, open(output_fn, mode="w") as fout:
    for line in fin:
        fout.write(re.sub(r"《[^》]+》|［[^］]+］|〔[^〕]+〕| [｜]", "", line))

冒頭と末尾の説明を取り除きます（何行取り除くかは目視で確認）。

In [8]:
!tail -n +22 text/sanshiro.stripruby.txt | head -n -14 > text/sanshiro.corpus.txt

head: illegal line count -- -14
tail: stdout: Broken pipe


これで、テキストファイルを扱う準備ができました。

### 形態素解析

形態素解析は、文章を一つ一つの形態素に分ける技術です。形態素は、「言葉が意味を持つまとまりの単語の最小単位」です。

spaCyで日本語の形態素解析モデル（`ja_ginza`）をロードして、形態素解析をしてみましょう。

In [11]:
import spacy

nlp = spacy.load("ja_ginza")
text = "これは文章です。"
doc = nlp(text)
for token in doc:
    print(token)

これ
は
文章
です
。


形態素解析の結果には、語の原形や品詞の情報も含まれます。

In [36]:
text = "昨日の天気は晴れでした。明日も晴れるでしょう。"
doc = nlp(text)
for token in doc:
    print(f"{token}\t{token.lemma_}\t{token.pos_}\t{token.tag_}")

昨日	昨日	NOUN	名詞-普通名詞-副詞可能
の	の	ADP	助詞-格助詞
天気	天気	NOUN	名詞-普通名詞-一般
は	は	ADP	助詞-係助詞
晴れ	晴れ	NOUN	名詞-普通名詞-一般
でし	です	AUX	助動詞
た	た	AUX	助動詞
。	。	PUNCT	補助記号-句点
明日	明日	NOUN	名詞-普通名詞-副詞可能
も	も	ADP	助詞-係助詞
晴れる	晴れる	VERB	動詞-一般
でしょう	です	AUX	助動詞
。	。	PUNCT	補助記号-句点


それでは、『三四郎』に出現する単語の頻度を数えてみましょう。

テキストファイルを読み込んで形態素解析を行います。

In [15]:
input_fn = "text/sanshiro.corpus.txt"
output_fn = "text/sanshiro.wakati.txt"

with open(input_fn, "r") as fin, open(output_fn, "w") as fout:
    for line in fin:
        tokens = [token.text for token in nlp(line.rstrip())]
        fout.write(' '.join(tokens) + "\n")

出力されたファイルを確認すると、分かち書きされていることが分かります。

次に、使用頻度の高い単語を見てみましょう。

In [27]:
from collections import Counter

# 分析対象とする品詞（内容語 - 名詞、動詞、形容詞）と不要語（ストップワード）を指定する
include_pos = ("NOUN", "VERB", "ADJ")
stopwords = ("する", "ある", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる")

# ファイルの読み込み、テキストを一行ずつ解析
all_tokens = []
with open(input_fn, "r") as f:
    for line in f:
        tokens = [token for token in nlp(line)]
        all_tokens.extend(tokens)
        
# 単語の頻度を数える
counter = Counter(token.lemma_ for token in all_tokens if token.pos_ in include_pos and token.lemma_ not in stopwords)

# 出現頻度top 10を出力する
for word, count in counter.most_common(10):
    print(f"{count:>5} {word}")

  614 言う
  385 女
  373 さん
  314 先生
  297 出る
  270 見る
  262 聞く
  262 人
  259 時
  247 思う


更に、単語の共起を見てみます。共起とは、どの語とどの語が一緒に使われているかを調べる方法です。

文章を文に分割し、同一文中に同時に出現する単語の組を数え上げることで分析します。

In [26]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
def extract_words(sent, pos_tags, stopwords):
    """
    分析対象の品詞であり、不要語ではない単語を抽出する
    """
    words = [token.lemma_ for token in sent if token.pos_ in pos_tags and token.lemma_ not in stopwords]
    return words

In [30]:
def count_cooccurrence(sents, token_length="{2,}"):
    """
    同じ文中に共起する単語を行列形式で列挙する
    """
    token_pattern = f"\\b\\w{token_length}\\b"
    count_model = CountVectorizer(token_pattern=token_pattern)
    
    X = count_model.fit_transform(sents)
    words = count_model.get_feature_names_out()
    word_counts = np.asarray(X.sum(axis=0).reshape(-1))
    
    X[X > 0] = 1 # 同じ共起が2以上出現しても1とする
    Xc = (X.T * X) # 共起行列を求めるための掛け算をする、csr形式の疎行列
    
    return words, word_counts, Xc, X

In [31]:
def find_sentence_by_cooccurrence(X, idxs):
    """
    指定された共起を含む文を見つける
    """
    occur_flags = (X[:, idxs[0]] > 0)
    for idx in idxs[1:]:
        occur_flags = occur_flags.multiply(X[:, idx] > 0)
    
    return occur_flags.nonzero()[0]

In [38]:
for sent in doc.sents:
    print(sent)

昨日の天気は晴れでした。
明日も晴れるでしょう。


### 係り受け解析

### 分類

### クラスタリング

## 可視化